In [5]:
#Variables in Python are memory references (pointers).
#They reference the object stored at a specific memory address
my_var = 10

#id() returns a base10 number which corresponds to the memory address of the given object
#hex() to return a base16
print(hex(id(my_var)))

0x101154210


In [9]:
my_var = 10
#In this circumstance, my_var_2 will point to the same object in memory.
#Their addresses will be the same.
my_var_2 = my_var

print(hex(id(my_var)))
print(hex(id(my_var_2)))


my_var = 20

#Memory addresses will now be different. my_var_2 will keep the original object
#my_var will now be a pointer to a different object of value 20
print(hex(id(my_var)))
print(hex(id(my_var_2)))

#Once both variables are reassigned, the reference count goes down to 0.
#If the reference count goes down to 0, the Python Memory Manager will drop the value from memory.


0x101154210
0x101154210
10
0x101154350
0x101154210


In [33]:
import sys
import ctypes

def ref_count(mem_address: int):
    return ctypes.c_long.from_address(mem_address).value

l = [1,2,3,4]
l2 = l

#sys.getrefcount() increases the total reference count by 1 -> 3 total in this case.
print(sys.getrefcount(l))

#ctypes.c_long.from_address(mem_address).value does not increase the reference count -> 2 total in this case.
#id(l) finished running first and releases its pointer, meaning that it does not increase the reference count.
print(ref_count(id(l)))


3
2


In [ ]:
import ctypes
import gc
import time

from multiprocessing.connection import wait
from signal import pause

def ref_count(mem_address: int):
    return ctypes.c_long.from_address(mem_address).value

def object_by_id(object_id):
    for obj in gc.get_objects():
        if id(obj)==object_id:
            return "Object exists"

    return "Not found"
         
#Define two classes to create a circular reference
#For GC demonstration   
class A:
    def __init__(self):
        self.b = B(self)
        print('A: self: {0}, b: {1}'.format(hex(id(self)), hex(id(self.b))))
class B:
    def __init__(self, a):
        self.a = a
        print('B: self: {0}, a: {1}'.format(hex(id(self)), hex(id(self.a))))

#Disable GC to allow for the experiment
gc.disable()

my_var = A()

print(hex(id(my_var)))

a_id = id(my_var)
b_id = id(my_var.b)

print(ref_count(a_id))
print(ref_count(b_id))

print(object_by_id(a_id))
print(object_by_id(b_id))

#Unset the variable
my_var = None

#Due to the circular reference, the objects still exist.
print(ref_count(a_id))
print(ref_count(b_id))

#Since GC is disabled, the objects still exist as well.
print(object_by_id(a_id))
print(object_by_id(b_id))

#Run GC manually.
gc.collect()

#No objects will be found afterwards.
print(object_by_id(a_id))
print(object_by_id(b_id))

time.sleep(5)

#ref_count will now return different values, as the memory addresses will eventually be occupied by something else.
print(ref_count(a_id))
print(ref_count(b_id))

In [38]:
#Dynamic vs Static typing

my_var = 5
print(type(my_var))

my_var = 'Hello'
print(type(my_var))

my_var = lambda x: 10 * 10
print(type(my_var))

#my_var is a reference without a static type. The object that my_var references has a type.

<class 'int'>
<class 'str'>
<class 'function'>


In [40]:
#Variable reassignment

my_a = 10
print(hex(id(my_a)))

#Prints a different address than the code above - integer cannot be changed, only reassigned
my_a = my_a + 3
print(hex(id(my_a)))

#Prints the same address
my_a = 10
my_b = 10
print(hex(id(my_a)))
print(hex(id(my_b)))

0x1030f4210
0x1030f4270
0x1030f4210
0x1030f4210


In [44]:
#Mutability and immutability

#tuples are immutable
#integers are also immutable, meaning that the tuple below and its contents cannot be changed.
t = (1,2,3)

#Lists are mutable. While the tuple containing them cannot be changed, the lists themselves can be.
t2 = ([1,2], [3,4])
print(t2)

#The object references in t2 did not change, but one of the objects was mutated.
t2[0].append(3)
print(t2)

4
([1, 2], [3, 4])
([1, 2, 3], [3, 4])


In [72]:
#Functions arguments and mutability

#Immutable objects are safe from unintended side-effects.
my_var = 'hello'

def changeString(s):
    s = s + ' there'
    return s
my_var_2 = changeString(my_var)


#Original my_var will not modified in this scope.
print(my_var)
print(my_var_2)


#Mutable objects are NOT safe.
list1 = [1,3,3,4]

def changeList(l):
    l.append(100)
changeList(list1)

#list1 has been mutated in this scope as well.
print(list1)

#Example with mutable inside of an immutable tuple
tup1 = ([1,2,3], [3,4,5])

def changeTuple(t):
    print('List address: ' + hex(id(t[0])))
    print('Tuple address: ' + hex(id(t)))
    print('-' * 50)
    t[0].append(300)

    #Here the memory address of the tuple changes, but not of the list.
    t = t + ([10000], 1)
    print('List address: ' + hex(id(t[0])))
    print('Tuple address: ' + hex(id(t)))
    print('-' * 50)

    t[1].append(5000)

changeTuple(tup1)

print('List address: ' + hex(id(tup1[0])))
print('Tuple address: ' + hex(id(tup1)))
print(tup1)
print('-' * 50)
print('Since the reference to the list was unchanged, the changes to it propagated to tup1, but the reassignment of t in changeTuple wasn\'t.')


hello
hello there
[1, 3, 3, 4, 100]
List address: 0x1087f5140
Tuple address: 0x109003c40
--------------------------------------------------
List address: 0x1087f5140
Tuple address: 0x10900d120
--------------------------------------------------
List address: 0x1087f5140
Tuple address: 0x109003c40
([1, 2, 3, 300], [3, 4, 5, 5000])
--------------------------------------------------
Since the reference to the list was unchanged, the changes to it propagated to tup1, but the reassignment of t in changeTuple wasn't.


In [3]:
#Shared references summary

a = 'hello'
b = a
c = 'hello'

print(hex(id(a)))
print(hex(id(b)))
print(hex(id(c)))
#Variables above all share the same reference. This is a result of interning.
#This is safe because they are immutable data types.
#While Python creates shared references for immutables automatically, it doesn't always do it -> interning is done in range [-5, 256]

print('-' * 50)

l1 = [1,2]
l2 = l1
l3 = [1,2]

print(hex(id(l1)))
print(hex(id(l2)))
print(hex(id(l3)))
#Only l1 and l2 share the same memory address.
#Python does not automatically create shared references for mutables.

print('-' * 50)


empty = None
empty2 = None

print(hex(id(empty)))
print(hex(id(empty2)))
#Python Memory Manager will always create a shared reference for None objects.
#This applies to any number of variables pointing to None throughout the lifetime of the application.

print('We can deduce is a variable is \'not set \' by is None: {0}.'.format(empty is None))

print(hex(id(None)))
print(type(None))




0x10366f0b0
0x10366f0b0
0x10366f0b0
--------------------------------------------------
0x103f13180
0x103f13180
0x103f1e300
--------------------------------------------------
0x101454ec0
0x101454ec0
We can deduce is a variable is 'not set ' by is None: True.
0x101454ec0
<class 'NoneType'>


In [12]:
#Retrieve useful info about a class
#help(int)

foo = int('1D', base=16)
print(foo)

def square(a, b):
    return a ** b

#Functions can be assigned just like any other object.
bar = square

print(bar(2, 2))

29
4


In [21]:
#Interning in range [-5, 256]

a = 256
b = 256
print(hex(id(a)))
print(hex(id(b)))
print(a is b)


c = 257
d = 257
print(hex(id(c)))
print(hex(id(d)))
print(c is d)

#It doesn't matter how the integer is created.
a = 10
b = int(10)
c = int('10')
d = int('1010', base=2)

print(a is b  is c is d)


0x1007960d0
0x1007960d0
True
0x10b9d5ad0
0x10b9d5870
False
True


In [28]:
#String interning
import sys

#This will be interned automatically. (Because it 'looks' like an identifier.)
a = 'hello'
b = 'hello'

print(a is b)
print(a == b)

#This will not be interned automatically.
a = 'for all the vain attempts'
b = 'for all the vain attempts'

print(a is b)
print(a == b)

#Here we force the interning.
a = sys.intern('for all the vain attempts')
b = sys.intern('for all the vain attempts')
c = 'for all the vain attempts'

print(a is b)
print(a == b)

print('This will only work if you intern every instance of that string. See \'a is c\': {0}'.format(a is c))

True
True
False
True
True
True
This will only work if you intern every instance of that string. See 'a is c': False


In [32]:
#Benchmarking string interning
import time
import sys

def compare_equals(n):
    a = 'I am a very long string' * 500
    b = 'I am a very long string' * 500
    for v in range(n):
        if a == b:
            pass

def compare_intern(n):
    a = sys.intern('I am a very long string' * 500)
    b = sys.intern('I am a very long string' * 500)

    for v in range(n):
        if a is b:
            pass


start_equals = time.perf_counter()
compare_equals(10000000)
end_equals = time.perf_counter()

start_intern = time.perf_counter()
compare_intern(10000000)
end_intern = time.perf_counter()

print(end_equals - start_equals)
print(end_intern - start_intern)

#Example results (in seconds):
#   4.481010666117072
#   0.17018029210157692


4.481010666117072
0.17018029210157692


In [36]:
#Peephole optimisations

#Constants get pre-calculated in bytecode (immutables, constant expressions)
def foo():
    a = 24 * 60
    b = 'hello'
    c = (1,2) * 5
    d = 'abc' * 50
    e = ['a', 'b'] * 5

foo.__code__.co_consts


def bar(e): 
    if e in [1,2,3]:
        pass

    if e in {1,2,3}:
        pass

bar.__code__.co_consts

(None, (1, 2, 3), frozenset({1, 2, 3}))

In [42]:
#Membership test performance
import numbers
import string
import time

charset = string.ascii_letters

char_list = list(charset)
char_tuple = tuple(charset)
char_set = set(charset)

def membership_test(n: int, container: list[str] | tuple[str] | set[str]):
    for v in range(n):
        if 'E' in container:
            pass

start_list = time.perf_counter()
membership_test(10000000, set(char_list))
end_list = time.perf_counter()

start_tuple = time.perf_counter()
membership_test(10000000, char_tuple)
end_tuple = time.perf_counter()

start_set = time.perf_counter()
membership_test(10000000, char_set)
end_set = time.perf_counter()

print(end_list - start_list)
print(end_tuple - start_tuple)
print(end_set - start_set)

#Example results
#    2.7387262501288205
#    2.605181416030973
#    0.21358341607265174

#Conclusion: Set membership tests are far more performant.

0.21509129204787314
2.610203708987683
0.21353366714902222
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
